In [1]:
from langchain.agents import AgentExecutor,ZeroShotAgent,Tool, tool

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [3]:
from langchain.callbacks.base import BaseCallbackHandler

class MyCustomHandler(BaseCallbackHandler):
    def on_agent_action(
        self, action, run_id, parent_run_id, **kwargs
    ):
        print("ON Agent action!", action)
        print("ON Agent action!", run_id)
        print("ON Agent action!", parent_run_id)

    def on_tool_start(self, serialized, input_str: str, **kwargs):
        """Run when tool starts running."""
        print("ON TOOL START!", kwargs.get('run_id'),kwargs.get('parent_run_id'))
        print("ON TOOL START!", input_str)

    def on_tool_end(self, output: str, **kwargs):
        """Run when tool ends running."""
        print("ON TOOL END!")
    def on_agent_finish(
        self,
        finish,
        *,
        run_id,
        parent_run_id,
        **kwargs,
    ):
        """Run on agent end."""
        print("ON Agent finish!", run_id)
        print("ON Agent finish!", parent_run_id)

In [4]:
def answer_metaphysics(*args, callbacks, **kwargs) -> str:
    """能回答关于玄学的问题"""
    print("ON TOOL running!",args)
    print("ON TOOL running!",callbacks.parent_run_id)
    print("ON TOOL running!",kwargs)
    return "玄学在东方文化中有着深厚的传统，被视为一种能够帮助人们理解宇宙和生命的智慧"

In [5]:
tools = [
    Tool(
        name="answer_metaphysics",
        func=answer_metaphysics,
        description="能回答关于玄学的问题",
    ),
]

In [6]:
agent_obj = ZeroShotAgent.from_llm_and_tools(llm,tools)

In [7]:
from langchain_core.runnables.config import RunnableConfig    
import uuid
runconfig = RunnableConfig(
    metadata={"test":"test"},
    callbacks=[MyCustomHandler()]
)

runconfig.items()

dict_items([('metadata', {'test': 'test'}), ('callbacks', [<__main__.MyCustomHandler object at 0x1269d5210>])])

In [8]:
agent = AgentExecutor.from_agent_and_tools(
        agent=agent_obj,
        tools=tools,
        verbose=True
)

In [9]:
agent.invoke({"input":"请问玄学是什么？"},runconfig)



> Entering new AgentExecutor chain...
ON Agent action! tool='answer_metaphysics' tool_input='玄学是研究超越感官经验范围的事物的学问，探讨宇宙、人生、灵魂等深层次的问题' log='我应该使用能回答关于玄学的工具来回答这个问题\nAction: answer_metaphysics\nAction Input: 玄学是研究超越感官经验范围的事物的学问，探讨宇宙、人生、灵魂等深层次的问题'
ON Agent action! f8e0de62-967a-41b2-af58-856baa281620
ON Agent action! None
我应该使用能回答关于玄学的工具来回答这个问题
Action: answer_metaphysics
Action Input: 玄学是研究超越感官经验范围的事物的学问，探讨宇宙、人生、灵魂等深层次的问题ON TOOL START! 5ce50d09-f0e4-48d6-8123-ffdb431225b3 f8e0de62-967a-41b2-af58-856baa281620
ON TOOL START! 玄学是研究超越感官经验范围的事物的学问，探讨宇宙、人生、灵魂等深层次的问题
ON TOOL running! ('玄学是研究超越感官经验范围的事物的学问，探讨宇宙、人生、灵魂等深层次的问题',)
ON TOOL running! 5ce50d09-f0e4-48d6-8123-ffdb431225b3
ON TOOL running! {}
ON TOOL END!

Observation: 玄学在东方文化中有着深厚的传统，被视为一种能够帮助人们理解宇宙和生命的智慧
Thought:ON Agent finish! f8e0de62-967a-41b2-af58-856baa281620
ON Agent finish! None
我现在明白了关于玄学的定义
Final Answer: 玄学是研究超越感官经验范围的事物的学问，探讨宇宙、人生、灵魂等深层次的问题，在东方文化中有着深厚的传统，被视为一种能够帮助人们理解宇宙和生命的智慧。

> Finished chain.


{'input': '请问玄学是什么？',
 'output': '玄学是研究超越感官经验范围的事物的学问，探讨宇宙、人生、灵魂等深层次的问题，在东方文化中有着深厚的传统，被视为一种能够帮助人们理解宇宙和生命的智慧。'}